In [123]:
import pandas as pd
import plotly.graph_objects as go

In [126]:
answerset_wa = pd.read_csv("/work/ThesisP1/data/answerset_wa.csv") # DRH entries for west asia

# reminder: upload west asia subsetting code
# reminder: decision for egypt

In [129]:
answerset_wa["branching_question"].value_counts()

branching_question
Non-elite (common people, general populace), Religious Specialists, Elite    33704
Religious Specialists, Elite                                                 20198
Non-elite (common people, general populace)                                   3874
Religious Specialists                                                         3186
Non-elite (common people, general populace), Religious Specialists            2693
Non-elite (common people, general populace), Elite                            1786
Elite                                                                          993
Name: count, dtype: int64

In [132]:
len(answerset_wa.loc[answerset_wa["poll_name"].str.contains("Place"), "entry_name"].unique())

73

In [76]:
# parent question IDs for "is supernatural monitoring present" in group, text, and place: 
## 2890, 4954, 6599, 8135, 8137, 8705
# answer value for absence of monitoring = 0

monitoring_wa = answerset_wa.loc[
    (answerset_wa["parent_question_id"].isin([2890, 4954, 6599, 8135, 8137, 8705])) &
    (answerset_wa["parent_answer_value"] != 0) &
    (answerset_wa["parent_answer_value"].notna())
]
monitoring_wa

,Unnamed: 0,region_id,region_name,region_description,world_region,question_name,answer,answer_value,notes,parent_question_id,...,branching_question,expert_id,expert_name,editor_id,editor_name,date_published,date_created,date_modified,gis_region,question_id
1119,8642,44.0,Levant,Levant,Southwest Asia,Supernatural beings care about murder of membe...,No,0.0,Many examples of Israelite gods killing out-gr...,2890.0,...,"Non-elite (common people, general populace)",207.0,Conn Herriott,14.0,Conn Herriott,2016-02-29 06:00:00.000000000,2019-07-09 17:41:46.459012864,2019-07-09 17:41:46.459032064,"MULTIPOLYGON(((35.270233154297 33.43276977539,...",2861.0
1121,8644,44.0,Levant,Levant,Southwest Asia,Supernatural beings care about murder of corel...,Yes,1.0,"However, at times this is condoned, ordered or...",2890.0,...,"Non-elite (common people, general populace)",207.0,Conn Herriott,14.0,Conn Herriott,2016-02-29 06:00:00.000000000,2019-07-09 17:41:46.436983040,2019-07-09 17:41:46.437001984,"MULTIPOLYGON(((35.270233154297 33.43276977539,...",2883.0
1122,8645,44.0,Levant,Levant,Southwest Asia,Supernatural beings care about honouring oaths:,Yes,1.0,"e.g. Proverbs 18:5. More broadly, the Israelit...",2890.0,...,"Non-elite (common people, general populace)",207.0,Conn Herriott,14.0,Conn Herriott,2016-02-29 06:00:00.000000000,2019-07-09 17:41:46.620729856,2019-07-09 17:41:46.620749056,"MULTIPOLYGON(((35.270233154297 33.43276977539,...",2887.0
1125,8648,44.0,Levant,Levant,Southwest Asia,Supernatural beings care about proper ritual o...,Yes,1.0,NaN,2890.0,...,"Non-elite (common people, general populace)",207.0,Conn Herriott,14.0,Conn Herriott,2016-02-29 06:00:00.000000000,2019-07-09 17:41:46.802340864,2019-07-09 17:41:46.802362880,"MULTIPOLYGON(((35.270233154297 33.43276977539,...",2898.0
1126,8649,44.0,Levant,Levant,Southwest Asia,Supernatural beings care about non-lethal figh...,Yes,1.0,Knight 2011: 142.,2890.0,...,"Non-elite (common people, general populace)",207.0,Conn Herriott,14.0,Conn Herriott,2016-02-29 06:00:00.000000000,2019-07-09 17:41:46.689686016,2019-07-09 17:41:46.689704960,"MULTIPOLYGON(((35.270233154297 33.43276977539,...",2902.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66393,369246,2477.0,"Zeyrek, Fatih, Istanbul",The monument is in the historical peninsula of...,Southwest Asia,Supernatural beings care about or expect perso...,Yes,1.0,NaN,8705.0,...,"Non-elite (common people, general populace), E...",1578.0,Theodoros Tzanatos,46.0,Megan Daniels,2024-10-01 18:26:54.937830912,2024-06-24 14:08:53.632827904,2024-10-01 18:26:54.937841920,MULTIPOLYGON(((28.956739191376677 41.019509799...,2939.0
66394,369247,2477.0,"Zeyrek, Fatih, Istanbul",The monument is in the historical peninsula of...,Southwest Asia,Supernatural beings care about honoring oaths:,Yes,1.0,NaN,8705.0,...,"Non-elite (common people, general populace), E...",1578.0,Theodoros Tzanatos,46.0,Megan Daniels,2024-10-01 18:26:54.937830912,2024-06-24 14:08:59.012805888,2024-10-01 18:26:54.937841920,MULTIPOLYGON(((28.956739191376677 41.019509799...,2887.0
66395,369248,2477.0,"Zeyrek, Fatih, Istanbul",The monument is in the historical peninsula of...,Southwest Asia,Supernatural beings care about or expect offer...,I don't know,-1.0,NaN,8705.0,...,"Non-elite (common people, general populace), E...",1578.0,Theodoros Tzanatos,46.0,Megan Daniels,2024-10-01 18:26:54.937830912,2024-06-24 14:09:09.209142016,2024-10-01 18:26:54.937841920,MULTIPOLYGON(((28.956739191376677 41.019509799...,5500.0
66396,369249,2477.0,"Zeyrek, Fatih, Istanbul",The monument is in the historical peninsula of...,Southwest Asia,Supernatural beings care about or expect maint...,Yes,1.0,NaN,8705.0,...,"Non-elite (common people, general populace), E...",1578.0,Theodoros Tzanatos,46.0,Megan Daniels,2024-10-01 18:26:54.937830912,2024-06-24 14:09:15.792995840,2024-10-01 18:26:54.937841920,MULTIPOLYGON(((28.956739191376677 41.019509799...,5512.0


In [135]:
len(monitoring_wa.loc[monitoring_wa["poll_name"].str.contains("Group"), "entry_name"].unique())

28

In [138]:
monitoring_wa["poll_name"].unique()

array(['Religious Group (v5)', 'Religious Group (v6)',
       'Religious Place (v1.2)', 'Religious Text (v1.0)',
       'Religious Place (v1.3)'], dtype=object)

In [114]:
# mix of human & gpt code

def display_drh(df):
    color_groups = {
        'Religious Group (v5)': "#00CC96",
        'Religious Group (v6)': "#00CC96",
        'Religious Text (v1.0)': "#E1B700",
        'Religious Place (v1.2)': "#AEDFF7",
        'Religious Place (v1.3)': "#AEDFF7"
    }

    # only interested in unique entries
    df_unique = df.drop_duplicates(subset="entry_name").copy()
    df_sorted = df_unique.sort_values("year_from").reset_index(drop=True)

    fig = go.Figure()

    for i, row in df_sorted.iterrows():
        entry = row["entry_name"]
        region = row["region_name"]
        poll = row["poll_name"]

        year_from = row["year_from"]
        year_to = row["year_to"] if (row["year_to"] - row["year_from"]) >= 20 else row["year_from"] + 20 # better visualiz
        color = color_groups.get(poll, "gray")

        fig.add_trace(go.Scatter(
            x=[year_from, year_to],
            y=[i, i],
            mode="lines",
            line=dict(color=color, width=2),
            hovertemplate=(
                f"<b>Entry:</b> {entry}<br>"
                f"<b>Region:</b> {region}<br>"
                f"<b>Years:</b> {year_from} – {year_to}<br>"
                f"<b>Poll:</b> {poll}"
            ),
            name="",
            showlegend=False
        ))

    # legend
    for label, color in {
        "Religious Group": "#00CC96",
        "Religious Text": "#E1B700",
        "Religious Place": "#AEDFF7"
    }.items():
        fig.add_trace(go.Scatter(
            x=[None], y=[None], mode='markers',
            marker=dict(color=color, size=10),
            name=label
        ))

    fig.update_layout(
        template="plotly_white",
        height=600,
        width=900,
        title="Timeline of Religious Units in West Asia & Egypt",
        xaxis=dict(
            title="Year",
            tickmode="linear",
            dtick=200,
            range=[df_sorted['year_from'].min() - 50, df_sorted['year_to'].max() + 50]
        ),
        yaxis=dict(
            showticklabels=False,
            title="",
            showgrid=False
        ),
        legend=dict(
            xanchor="left",
            yanchor="top",
            x = 0,
            y = 1
        )
    )

    fig.show()


In [117]:
display_drh(monitoring_wa)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5d7c084c-ccac-4a11-8892-8e2f6a069ef0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>